In [1]:
import os
import pandas as pd
import numpy as np
import joblib
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

# Lectura y preprocesamiento

In [11]:
# Carga el dataset
df_og = pd.read_csv('loan/loan.csv')

 # Mapea el target
status_map = {
     "Fully Paid": 0,
     "Charged Off": 1,
     "Late (31-120 days)": 1,
     "Default": 1,
     "Does not meet the credit policy. Status:Fully Paid": 0,
     "Does not meet the credit policy. Status:Charged Off": 1
 }
df_og['target'] = df_og['loan_status'].map(status_map)

C:\Users\User\AppData\Local\Temp\ipykernel_17916\3215090333.py:2: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df_og = pd.read_csv('loan/loan.csv')


In [13]:
import pandas as pd
  # o como sea que obtengas tu DataFrame
df = df_og[df_og['target'].notna()].copy()

In [154]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,target
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,1075269,1311441,5000.0,5000.0,5000.0,36 months,7.90,156.46,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


## Explora el balance de clases

In [14]:
print(df['target'].value_counts())
print(df['target'].value_counts(normalize=True))

target
0.0    209711
1.0     58819
Name: count, dtype: int64
target
0.0    0.780959
1.0    0.219041
Name: proportion, dtype: float64


## Preprocesamiento

In [15]:
df_varibales_categoricas = df.select_dtypes(include=['object']).astype("category")

In [16]:
null_counts = df_varibales_categoricas.isna().sum()
print(null_counts[null_counts > 15])

emp_title                     14958
emp_length                    10721
desc                         176559
title                            16
earliest_cr_line                 29
last_pymnt_d                    668
next_pymnt_d                 252971
last_credit_pull_d               24
verification_status_joint    268527
dtype: int64


In [17]:
df["earliest_cr_line"]

0         Jan-1985
1         Apr-1999
2         Nov-2001
3         Feb-1996
5         Nov-2004
            ...   
887351    Aug-1990
887364    Nov-1975
887366    Feb-2011
887369    Sep-2002
887371    Jun-2002
Name: earliest_cr_line, Length: 268530, dtype: object

In [18]:
df["last_credit_pull_d"]

0         Jan-2016
1         Sep-2013
2         Jan-2016
3         Jan-2015
5         Sep-2015
            ...   
887351    Jan-2016
887364    Jun-2015
887366    Dec-2015
887369    Nov-2015
887371    Jan-2016
Name: last_credit_pull_d, Length: 268530, dtype: object

In [19]:
df["last_pymnt_d"]

0         Jan-2015
1         Apr-2013
2         Jun-2014
3         Jan-2015
5         Jan-2015
            ...   
887351    Aug-2015
887364    Jul-2015
887366    Dec-2015
887369    Jun-2015
887371    Aug-2015
Name: last_pymnt_d, Length: 268530, dtype: object

se eliminan todas estas columnas puesto que tienen más de 10 mil nulos y las otras 3 no son significativas y agregarian mucha complejidad al ser fechas

In [20]:
# Remove columns with many null values and date columns
columns_to_remove = ['earliest_cr_line', 'last_credit_pull_d', 'last_pymnt_d', "emp_length", "next_pymnt_d", "verification_status_joint", "desc", "emp_title", "title"]
df = df.drop(columns=columns_to_remove, errors='ignore')

In [21]:
df_varibales_categoricas = df.select_dtypes(include=['object']).astype("category")

In [22]:
df_varibales_categoricas.head()

,term,grade,sub_grade,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,zip_code,addr_state,initial_list_status,application_type
0,36 months,B,B2,RENT,Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,credit_card,860xx,AZ,f,INDIVIDUAL
1,60 months,C,C4,RENT,Source Verified,Dec-2011,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,car,309xx,GA,f,INDIVIDUAL
2,36 months,C,C5,RENT,Not Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,small_business,606xx,IL,f,INDIVIDUAL
3,36 months,C,C1,RENT,Source Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,other,917xx,CA,f,INDIVIDUAL
5,36 months,A,A4,RENT,Source Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,wedding,852xx,AZ,f,INDIVIDUAL


In [23]:
df_varibales_categoricas["application_type"].value_counts()

application_type
INDIVIDUAL    268527
JOINT              3
Name: count, dtype: int64

se elimina porque practicamente todos los valores son iguales, por lo que no tendrá significancia para el modelo

In [24]:
df = df.drop(columns=["application_type"], errors='ignore')

A el resto de columnas categoricas se les aplica one hot encoder para incluirlas en el analisis de significancia de las variables para el modelo

In [25]:
# Elimina columnas problemáticas
cols_to_drop = ['id', 'member_id', 'url', 'desc', 'title', 'zip_code', 'loan_status', 'emp_title']
df = df.drop(columns=cols_to_drop, errors='ignore')

In [26]:
# Get categorical columns from df (excluding certain columns we don't want to encode)
df_varibales_categoricas = df.select_dtypes(include=['object'])
categorical_columns = df_varibales_categoricas.columns
categorical_columns = [col for col in categorical_columns]

# Apply one-hot encoding
df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

In [27]:
df_varibales_categoricas 

,term,grade,sub_grade,home_ownership,verification_status,issue_d,pymnt_plan,purpose,addr_state,initial_list_status
0,36 months,B,B2,RENT,Verified,Dec-2011,n,credit_card,AZ,f
1,60 months,C,C4,RENT,Source Verified,Dec-2011,n,car,GA,f
2,36 months,C,C5,RENT,Not Verified,Dec-2011,n,small_business,IL,f
3,36 months,C,C1,RENT,Source Verified,Dec-2011,n,other,CA,f
5,36 months,A,A4,RENT,Source Verified,Dec-2011,n,wedding,AZ,f
...,...,...,...,...,...,...,...,...,...,...
887351,36 months,D,D2,MORTGAGE,Verified,Jan-2015,n,medical,CO,f
887364,36 months,A,A1,RENT,Not Verified,Jan-2015,n,debt_consolidation,FL,w
887366,36 months,D,D3,RENT,Source Verified,Jan-2015,n,debt_consolidation,FL,f
887369,36 months,B,B1,MORTGAGE,Verified,Jan-2015,n,car,CA,f


In [28]:
df_encoded.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,...,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,initial_list_status_w
0,5000.0,5000.0,4975.0,10.65,162.87,24000.0,27.65,0.0,1.0,NaN,...,False,False,False,False,False,False,False,False,False,False
1,2500.0,2500.0,2500.0,15.27,59.83,30000.0,1.00,0.0,5.0,NaN,...,False,False,False,False,False,False,False,False,False,False
2,2400.0,2400.0,2400.0,15.96,84.33,12252.0,8.72,0.0,2.0,NaN,...,False,False,False,False,False,False,False,False,False,False
3,10000.0,10000.0,10000.0,13.49,339.31,49200.0,20.00,0.0,1.0,35.0,...,False,False,False,False,False,False,False,False,False,False
5,5000.0,5000.0,5000.0,7.90,156.46,36000.0,11.20,0.0,3.0,NaN,...,False,False,False,False,False,False,False,False,False,False


## Imputacion de nulos

In [29]:
# Imputa nulos
df = df.fillna(df.median(numeric_only=True))

In [30]:
# Imputa nulos
df_encoded = df_encoded.fillna(df_encoded.median(numeric_only=True))

# Analisis de las variables más significativas

In [34]:
X = df_encoded.drop(columns=['target'], errors='ignore')
y = df['target']

## Metodo kbest

In [35]:
# SelectKBest
selector = SelectKBest(f_classif, k=7)
selector.fit(X, y)

# Obtener scores
scores = pd.DataFrame({
    'Feature': X.columns,
    'Score': selector.scores_
})
scores = scores.sort_values('Score', ascending=False)

print("Importancia de características:")
print(scores)

Importancia de características:
              Feature         Score
20    total_rec_prncp  83367.562104
25    last_pymnt_amnt  54935.222040
23         recoveries  48290.312606
18        total_pymnt  46675.895755
19    total_pymnt_inv  45848.611281
..                ...           ...
161  issue_d_Mar-2013      0.011983
224     addr_state_IA      0.010454
164  issue_d_May-2008      0.001830
235     addr_state_MN      0.001272
28        policy_code           NaN

[264 rows x 2 columns]


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [28] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [37]:
scores.head(10)

,Feature,Score
20,total_rec_prncp,83367.562104
25,last_pymnt_amnt,54935.222040
23,recoveries,48290.312606
18,total_pymnt,46675.895755
19,total_pymnt_inv,45848.611281
16,out_prncp,37543.371479
17,out_prncp_inv,37542.785395
24,collection_recovery_fee,20652.987243
3,int_rate,18640.769072
49,term_ 60 months,7765.671379


In [38]:
columnas_seleccionadas = selector.get_support()
nombres_columnas = X.columns[columnas_seleccionadas]
X = X[nombres_columnas]
X.head()

,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,recoveries,last_pymnt_amnt
0,0.0,0.0,5861.071414,5831.78,5000.00,0.00,171.62
1,0.0,0.0,1008.710000,1008.71,456.46,117.08,119.66
2,0.0,0.0,3003.653644,3003.65,2400.00,0.00,649.91
3,0.0,0.0,12226.302212,12226.30,10000.00,0.00,357.48
5,0.0,0.0,5631.377753,5631.38,5000.00,0.00,161.03


In [39]:
print(df_encoded.groupby('target')[nombres_columnas].count())

        out_prncp  out_prncp_inv  total_pymnt  total_pymnt_inv  \
target                                                           
0.0        209711         209711       209711           209711   
1.0         58819          58819        58819            58819   

        total_rec_prncp  recoveries  last_pymnt_amnt  
target                                                
0.0              209711      209711           209711  
1.0               58819       58819            58819  


lo recomendado es quedarnos con las 7 primeras variables, puesto que a partir de la octava cae considereablemente el score (significancia de la variable), por lo cual, a partir de esta no serian muy explicativas.

## Entrenamiento de modelos

In [56]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import shap

# Usa el DataFrame ya codificado y sin nulos
X = df_encoded.drop(columns=['target'], errors='ignore')
y = df['target']

# Selección de variables (elige el número que prefieras, aquí 7)
k = 7
selector = SelectKBest(score_func=f_classif, k=k)
X_new = selector.fit_transform(X, y)
mask = selector.get_support()
selected_features = X.columns[mask]
print("Variables seleccionadas:", selected_features)

# Entrenamiento con las variables seleccionadas
X_selected = X[selected_features]
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Asegúrate de que las columnas y el orden coincidan
X_test = X_test[X_train.columns]

# Gráfica SHAP (con verificación de aditividad desactivada para evitar errores)
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test, check_additivity=False)
shap.summary_plot(shap_values, X_test, feature_names=selected_features)

KeyboardInterrupt: 

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

param_grid = [
    {
        'penalty': ['l1'], 
        'solver': ['liblinear', 'saga'],
        'C': [0.01, 0.1, 1, 10, 100]
    },
    {
        'penalty': ['l2'],
        'solver': ['lbfgs', 'liblinear', 'saga'],
        'C': [0.01, 0.1, 1, 10, 100]
    }
]

log_reg = LogisticRegression(max_iter=1000, random_state=42)

grid_search = GridSearchCV(
    estimator=log_reg, 
    param_grid=param_grid, 
    cv=5, 
    scoring='roc_auc', 
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000, random_state=42),
             n_jobs=-1,
             param_grid=[{'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1'],
                          'solver': ['liblinear', 'saga']},
                         {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l2'],
                          'solver': ['lbfgs', 'liblinear', 'saga']}],
             scoring='roc_auc')

In [57]:
model = grid_search.best_estimator_

In [58]:
y_pred_prob = model.predict_proba(X_test)[:,1]
y_pred = model.predict(X_test)

print("AUC:", roc_auc_score(y_test, y_pred_prob))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

AUC: 0.9944030211277125
              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99     42051
         1.0       0.95      0.94      0.95     11655

    accuracy                           0.98     53706
   macro avg       0.97      0.97      0.97     53706
weighted avg       0.98      0.98      0.98     53706

[[41519   532]
 [  656 10999]]
